In [1]:
# Librerie
import os
import pandas as pd
import numpy as np
import scipy as sp
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import random
sns.set_style('whitegrid')
import itertools

from math import factorial
from IPython.core.display import display, HTML

In [2]:
# Funzioni


In [77]:
df = pd.read_csv('../../src/clean_csv/clean_dataframe.csv', sep='\t')
df.columns = ['index', 'id_par','label', 'statement', 'subject', 'speaker', 
              'job', 'state', 'party', 'context']
df = df.drop(['index'], axis=1)

df = df[~df['id_par'].isin(['1779.json'])]
# replace string
t_repl = {'barely-true': 'BARELYTRUE',
          'false': 'FALSE',
          'half-true': 'HALFTRUE',
          'mostly-true': 'MOSTLYTRUE',
          'pants-fire' : 'LIE',
          'true' : 'TRUE'
        }

def f(l):
    return [ t_repl.get(item,item) for item in l ]
df['label'] = df.label.str.split('<').apply(f).str.join('')

df['name'] = '-1'
display(df.head())
display(df.shape)

,id_par,label,statement,subject,speaker,job,state,party,context,name
0,10540.json,HALFTRUE,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,-1
1,324.json,MOSTLYTRUE,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,Denver,-1
2,1123.json,FALSE,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,a news release,-1
3,9028.json,HALFTRUE,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN,-1
4,12465.json,TRUE,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,a an online opinion-piece,-1


(12785, 10)

In [78]:
# Prendo solo Democratici, Repubblicani e Bernie
sub_df = df[(df.party == "democrat") | (df.party == "republican")]

display(sub_df.head())
display(sub_df.shape)


,id_par,label,statement,subject,speaker,job,state,party,context,name
0,10540.json,HALFTRUE,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,-1
1,324.json,MOSTLYTRUE,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,Denver,-1
3,9028.json,HALFTRUE,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN,-1
4,12465.json,TRUE,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,a an online opinion-piece,-1
5,2342.json,BARELYTRUE,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,a press release.,-1


(9798, 10)

In [79]:
filtered = pd.DataFrame(columns=sub_df.columns)
categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
docs_per_cat = 100

for cat in categories:
    # Prendo i primi docs_per_cat per ogni label
    sub = sub_df[sub_df['label'] == cat].head(docs_per_cat)
    assert sub.shape[0] == docs_per_cat, 'Non ho abbastanza docs per tutte le categorie'
    # Nomino i documenti
    sub['name'] = [cat + '_doc' + str(x + 1) for x in range(len(sub))]
    # Concateno il tutto
    filtered  = pd.concat([filtered, sub])
    
# Compongo il Json con i campi necessari
filtered['js'] = [ {'id_par': filtered.iloc[x]['id_par'],
                    'name': filtered.iloc[x]['name'],
                    'statement': (filtered.iloc[x]['statement']).replace("'", ""),
                    'speaker': filtered.iloc[x]['speaker'],
                    'job': filtered.iloc[x]['job'],
                    'context': filtered.iloc[x]['context'],
                    }    
                  for x in range(len(filtered))]
filtered = filtered[['name', 'js']]

# Aggiungo Gold High | Low
high = {'id_par':'HIGH',
        'name':'HIGH',
        'statement':'Yes, We can!',
        'speaker':'Barack Obama',
        'job':'Former U.S. President',
        'context':'Public appearance',
        
        }
low = {'id_par':'LOW',
        'name':'LOW',
        'statement':'if a were not a sumo wrestler, I would be a football player.',
        'speaker':'Abraham Lincoln',
        'job':'Former U.S. President',
        'context':'Public appearance',
        }


filtered = filtered.append({'name' : 'HIGH', 'js': high}, ignore_index = True)
filtered = filtered.append({'name' : 'LOW', 'js': low}, ignore_index = True)
display(filtered.tail())


,name,js
597,TRUE_doc98,"{'id_par': '4875.json', 'name': 'TRUE_doc98', ..."
598,TRUE_doc99,"{'id_par': '9576.json', 'name': 'TRUE_doc99', ..."
599,TRUE_doc100,"{'id_par': '5716.json', 'name': 'TRUE_doc100',..."
600,HIGH,"{'id_par': 'HIGH', 'name': 'HIGH', 'statement'..."
601,LOW,"{'id_par': 'LOW', 'name': 'LOW', 'statement': ..."


In [80]:
categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
total_docs = docs_per_cat * len(categories)

docs_each_hit = 6
gold_each_hit = 2
actual_docs_each_hit = docs_each_hit + gold_each_hit

assert total_docs % docs_each_hit == 0, 'Errore1'
assert docs_each_hit % len(categories) == 0, 'Errore2'

repetitions_each_doc = 10

# shuffle della riga o rifaccio campione dall'inizio, per ora shuffle della riga.
df = pd.DataFrame(columns = ['d{}'.format(x + 1) for x in range(actual_docs_each_hit)])

doc_pool = {"{}".format(categories[j]):["{}_doc{}".format(categories[j], i + 1) for i in range(docs_per_cat)] for j in range(len(categories)) }

def sample_per_cat(pool, categories):
    sample = ['HIGH', 'LOW']
    for j in categories:
        selected = random.choice(pool[j])
        sample.append(selected)
        # remove
        lst = doc_pool[j]
        lst.remove(selected)
        doc_pool[j] = lst
    return sample

for i in range( int(total_docs / docs_each_hit) ):
    hit_sample = []
    for z in range( int(docs_each_hit / len(categories))  ):
        choice = sample_per_cat(pool = doc_pool, categories=categories)
        hit_sample.append(choice)
    # flatten list
    hit_sample = list(itertools.chain.from_iterable(hit_sample))
    # repeat the hits j times
    for j in range(repetitions_each_doc):
        random.shuffle(hit_sample)
        df.loc[len(df)] = hit_sample
    
display(doc_pool)
display(df.head())
display(filtered.head())

display('HITs: {}'.format(df.shape[0]))
display( 'judgments:{}'.format(df.shape[0]*docs_each_hit) )
price = 0.2
display( 'cost:{}'.format(df.shape[0]*price) )

{'BARELYTRUE': [],
 'FALSE': [],
 'HALFTRUE': [],
 'LIE': [],
 'MOSTLYTRUE': [],
 'TRUE': []}

,d1,d2,d3,d4,d5,d6,d7,d8
0,HIGH,HALFTRUE_doc36,FALSE_doc8,LOW,LIE_doc42,TRUE_doc27,BARELYTRUE_doc24,MOSTLYTRUE_doc87
1,BARELYTRUE_doc24,HALFTRUE_doc36,HIGH,LIE_doc42,LOW,FALSE_doc8,MOSTLYTRUE_doc87,TRUE_doc27
2,HIGH,LIE_doc42,BARELYTRUE_doc24,LOW,MOSTLYTRUE_doc87,HALFTRUE_doc36,FALSE_doc8,TRUE_doc27
3,LOW,FALSE_doc8,TRUE_doc27,MOSTLYTRUE_doc87,HIGH,LIE_doc42,HALFTRUE_doc36,BARELYTRUE_doc24
4,LIE_doc42,HALFTRUE_doc36,BARELYTRUE_doc24,HIGH,MOSTLYTRUE_doc87,TRUE_doc27,FALSE_doc8,LOW


,name,js
0,LIE_doc1,"{'id_par': '5210.json', 'name': 'LIE_doc1', 's..."
1,LIE_doc2,"{'id_par': '12848.json', 'name': 'LIE_doc2', '..."
2,LIE_doc3,"{'id_par': '10829.json', 'name': 'LIE_doc3', '..."
3,LIE_doc4,"{'id_par': '162.json', 'name': 'LIE_doc4', 'st..."
4,LIE_doc5,"{'id_par': '3864.json', 'name': 'LIE_doc5', 's..."


'HITs: 1000'

'judgments:6000'

'cost:200.0'

In [81]:
units = df.copy()

keys = filtered['name'].values
vals = filtered['js'].values
testo_dict = {keys[i]:vals[i] for i in range(len(filtered))}
# Mappo
for col in units.columns.values:
    units[col] = [testo_dict.get(item,item) for item in units[col] ]
    if(str(units[col]) == "HIGH"):
        display(units[col])
    
# Salvo posizione High e Low
highs = []
lows = []
for i in range(len(units)):
    sub = list(df.iloc[i].values)
    h = sub.index('HIGH') + 1
    l = sub.index('LOW') + 1
    highs.append(h)
    lows.append(l)
# Aggiunta a DataFrame
df['high'] = highs
df['low'] = lows
df['id_unit'] = ['unit_' + str(x) for x in range(len(units))]

display(units.head())
#display(filtered.head())
#display(df.head())

df.to_csv('../../Units/David/SCHEMAunits.csv', index = False, header = True)

,d1,d2,d3,d4,d5,d6,d7,d8
0,"{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '5297.json', 'name': 'HALFTRUE_doc3...","{'id_par': '8187.json', 'name': 'FALSE_doc8', ...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ...","{'id_par': '1163.json', 'name': 'LIE_doc42', '...","{'id_par': '2550.json', 'name': 'TRUE_doc27', ...","{'id_par': '13370.json', 'name': 'BARELYTRUE_d...","{'id_par': '3520.json', 'name': 'MOSTLYTRUE_do..."
1,"{'id_par': '13370.json', 'name': 'BARELYTRUE_d...","{'id_par': '5297.json', 'name': 'HALFTRUE_doc3...","{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '1163.json', 'name': 'LIE_doc42', '...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ...","{'id_par': '8187.json', 'name': 'FALSE_doc8', ...","{'id_par': '3520.json', 'name': 'MOSTLYTRUE_do...","{'id_par': '2550.json', 'name': 'TRUE_doc27', ..."
2,"{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '1163.json', 'name': 'LIE_doc42', '...","{'id_par': '13370.json', 'name': 'BARELYTRUE_d...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ...","{'id_par': '3520.json', 'name': 'MOSTLYTRUE_do...","{'id_par': '5297.json', 'name': 'HALFTRUE_doc3...","{'id_par': '8187.json', 'name': 'FALSE_doc8', ...","{'id_par': '2550.json', 'name': 'TRUE_doc27', ..."
3,"{'id_par': 'LOW', 'name': 'LOW', 'statement': ...","{'id_par': '8187.json', 'name': 'FALSE_doc8', ...","{'id_par': '2550.json', 'name': 'TRUE_doc27', ...","{'id_par': '3520.json', 'name': 'MOSTLYTRUE_do...","{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '1163.json', 'name': 'LIE_doc42', '...","{'id_par': '5297.json', 'name': 'HALFTRUE_doc3...","{'id_par': '13370.json', 'name': 'BARELYTRUE_d..."
4,"{'id_par': '1163.json', 'name': 'LIE_doc42', '...","{'id_par': '5297.json', 'name': 'HALFTRUE_doc3...","{'id_par': '13370.json', 'name': 'BARELYTRUE_d...","{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '3520.json', 'name': 'MOSTLYTRUE_do...","{'id_par': '2550.json', 'name': 'TRUE_doc27', ...","{'id_par': '8187.json', 'name': 'FALSE_doc8', ...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ..."


In [1]:
## Genero i docs
sep = '=++='

## Salvo in .txt High e Low
id_par = 'HIGH'
name = 'HIGH'
statement = 'Yes, We can!'
speaker = 'Barack Obama'
job = 'Former U.S. President'
context ='Public appearance'
my_str = id_par + sep + name + sep + statement + sep + speaker + sep + job + sep + context

text_file = open("../../Units/David/remote-units/{}.txt".format(name), "w")
text_file.write(my_str)
text_file.close()

id_par = 'LOW'
name = 'LOW'
statement = 'if a were not a sumo wrestler, I would be a football player.'
speaker = 'Abraham Lincoln'
job = 'Former U.S. President'
context ='Public appearance'
my_str = id_par + sep + name + sep + statement + sep + speaker + sep + job + sep + context

text_file = open("../../Units/David/remote-units/{}.txt".format(name), "w")
text_file.write(my_str)
text_file.close()

## Other Statements
for unit in range(len(units)):
    print('{}, '.format(unit), end = '')
    for doc in range(1, 9):
        # id_par, name, statement, speaker, job, context
        dic = units.loc[unit]["d" + str(doc)]
        id_par = dic['id_par']
        name = dic['name']
        statement = dic['statement']
        speaker = dic['speaker']
        job = str(dic['job'])
        context = str(dic['context'])
        
        sep = "=++="
        my_str = id_par + sep + name + sep + statement + sep + speaker + sep + job + sep + context
        #display(my_str)
        text_file = open("../../Units/David/remote-units/{}.txt".format(name), "w")
        text_file.write(my_str)
        text_file.close()

NameError: name 'units' is not defined